In [1]:
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df_symptoms = pd.read_csv('test_data/results_symptoms.csv')

In [4]:
len(df_symptoms)

93995

In [5]:
df_symptoms.columns

Index(['R', 'temperature', 'high_risk_exposure_occupation',
       'high_risk_interactions', 'labored_respiration', 'rhonchi', 'cough',
       'fever', 'sob', 'diarrhea', 'fatigue', 'headache', 'loss_of_smell',
       'loss_of_taste', 'runny_nose', 'muscle_sore', 'sore_throat', 'wheezes'],
      dtype='object')

In [55]:
df_symptoms.describe()

,R,temperature,high_risk_exposure_occupation,high_risk_interactions,labored_respiration,rhonchi,cough,fever,sob,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,wheezes,id
count,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000
mean,0.013969,36.794765,0.073903,0.205514,0.007959,0.303590,0.062194,0.028477,0.030302,0.020222,0.065147,0.057786,0.007068,0.007206,0.036873,0.036136,0.062977,0.346879,46997.000000
std,0.117362,0.205696,0.261379,0.346630,0.063993,0.229147,0.241490,0.144638,0.171230,0.140620,0.246555,0.233112,0.083689,0.084499,0.188264,0.186448,0.242686,0.257399,27134.163613
min,0.000000,33.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,36.794765,0.000000,0.000000,0.000000,0.303590,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.346879,23498.500000
50%,0.000000,36.794765,0.000000,0.000000,0.000000,0.303590,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.346879,46997.000000
75%,0.000000,36.800000,0.000000,0.205514,0.007959,0.303590,0.000000,0.028477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.346879,70495.500000
max,1.000000,39.600000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,93994.000000


In [7]:
df_symptoms.describe()

,R,temperature,high_risk_exposure_occupation,high_risk_interactions,labored_respiration,rhonchi,cough,fever,sob,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,wheezes
count,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000
mean,0.013969,36.794765,0.073903,0.205514,0.007959,0.303590,0.062194,0.028477,0.030302,0.020222,0.065147,0.057786,0.007068,0.007206,0.036873,0.036136,0.062977,0.346879
std,0.117362,0.205696,0.261379,0.346630,0.063993,0.229147,0.241490,0.144638,0.171230,0.140620,0.246555,0.233112,0.083689,0.084499,0.188264,0.186448,0.242686,0.257399
min,0.000000,33.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,36.794765,0.000000,0.000000,0.000000,0.303590,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.346879
50%,0.000000,36.794765,0.000000,0.000000,0.000000,0.303590,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.346879
75%,0.000000,36.800000,0.000000,0.205514,0.007959,0.303590,0.000000,0.028477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.346879
max,1.000000,39.600000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score

In [9]:
#df_symptoms.dropna(inplace=True) # leaves 1946

In [10]:
# normalize the temperaature
#df_symptoms['temperature'] = (df_symptoms['temperature'] - df_symptoms['temperature'].min()) / (df_symptoms['temperature'].max() - df_symptoms['temperature'].min())

In [11]:
df_symptoms.head(5)

,R,temperature,high_risk_exposure_occupation,high_risk_interactions,labored_respiration,rhonchi,cough,fever,sob,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,wheezes
0,0,36.95,1.000000,0.205514,0.000000,0.00000,1.0,0.028477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,36.75,0.000000,0.205514,0.000000,0.00000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0,36.95,0.073903,0.205514,0.007959,0.30359,0.0,0.028477,0.030302,0.020222,0.065147,0.057786,0.007068,0.007206,0.036873,0.036136,0.062977,0.346879
3,0,36.75,1.000000,1.000000,0.000000,0.30359,1.0,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.346879
4,0,36.25,0.000000,0.205514,0.000000,0.00000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
#df_symptoms[df_symptoms['R'] == 1]
df_symptoms["id"] = df_symptoms.index

In [13]:
df_symptoms.iloc[:,2:-1].head()

,high_risk_exposure_occupation,high_risk_interactions,labored_respiration,rhonchi,cough,fever,sob,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,wheezes
0,1.000000,0.205514,0.000000,0.00000,1.0,0.028477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.205514,0.000000,0.00000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.073903,0.205514,0.007959,0.30359,0.0,0.028477,0.030302,0.020222,0.065147,0.057786,0.007068,0.007206,0.036873,0.036136,0.062977,0.346879
3,1.000000,1.000000,0.000000,0.30359,1.0,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.346879
4,0.000000,0.205514,0.000000,0.00000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
len(df_symptoms)

93995

In [15]:
#df_symptoms.head()

In [16]:
symptom_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier())
    ])

In [17]:
# don't include the ID as a feature
symptom_clf.fit(df_symptoms.iloc[:,2:-1], df_symptoms['R'])

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', RandomForestClassifier())])

In [18]:
symptom_clf['scaler'].scale_

array([0.26137728, 0.34662841, 0.06399237, 0.22914554, 0.24148845,
       0.14463698, 0.17122923, 0.14061935, 0.24655342, 0.23311067,
       0.08368819, 0.08449878, 0.18826286, 0.18644701, 0.24268503,
       0.25739785])

In [19]:
symptom_clf['clf'].feature_importances_

array([0.06369386, 0.10777984, 0.04507163, 0.05601844, 0.05682146,
       0.08790324, 0.0542134 , 0.04606716, 0.06472785, 0.06005754,
       0.07161652, 0.05154851, 0.05600384, 0.04974493, 0.07010412,
       0.05862766])

In [20]:
feature_df = pd.DataFrame({'feature':df_symptoms.columns[2:-1], 
                           'importance': symptom_clf['clf'].feature_importances_})

In [21]:
pysqldf("SELECT * FROM feature_df ORDER BY importance DESC")

,feature,importance
0,high_risk_interactions,0.107780
1,fever,0.087903
2,loss_of_smell,0.071617
3,sore_throat,0.070104
4,fatigue,0.064728
5,high_risk_exposure_occupation,0.063694
6,headache,0.060058
7,wheezes,0.058628
8,cough,0.056821
9,rhonchi,0.056018


In [22]:
symptom_scores = cross_val_score(symptom_clf, df_symptoms.iloc[:,2:], df_symptoms['R'], cv=5)

In [23]:
print("Accuracy: %0.2f, StDev: %0.2f)" % (symptom_scores.mean(), symptom_scores.std()))

Accuracy: 0.32, StDev: 0.35)


In [24]:
test_predicted_proba = symptom_clf.predict_proba(df_symptoms.iloc[:, 2:-1])

In [25]:
test_predicted = symptom_clf.predict(df_symptoms.iloc[:, 2:-1])

In [26]:
df_pred = pd.DataFrame({"p_cat":test_predicted, "p_prob":[item[1] for item in test_predicted_proba], "p_id":df_symptoms.id })

In [27]:
pysqldf("SELECT * FROM df_pred ORDER BY p_prob DESC LIMIT 5")

,p_cat,p_prob,p_id
0,1,0.948333,2398
1,1,0.948333,21524
2,1,0.940000,982
3,1,0.940000,7102
4,1,0.940000,12258


In [53]:
symptom_clf.predict_proba([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

array([[0.98725932, 0.01274068]])

In [30]:
symptom_clf.predict_proba([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])

array([[0.69, 0.31]])

In [31]:
symptom_clf.predict_proba([list(df_symptoms.iloc[123][2:-1])])

array([[0.9284916, 0.0715084]])

In [47]:
symptom_clf.predict_proba([[1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

array([[0.98198261, 0.01801739]])

In [48]:
y_pred = symptom_clf.predict(df_symptoms.iloc[:, 2:-1])

In [51]:
len(y_pred), y_pred.sum()

(93995, 310)

### Accuracy: 

How often did the predicted value match the actual value

### Precision: 

When you predicted a positive result, how often was the actual value positive?

### Recall: 

When the actual value was positive, how often did you predict a positive value?

### Visual:

https://en.wikipedia.org/wiki/Precision_and_recall

In [39]:
precision, recall, fscore, train_support = score(df_symptoms['R'], y_pred, pos_label=1, average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==df_symptoms['R']).sum()/len(y_pred), 3)))

Precision: 0.926 / Recall: 0.219 / Accuracy: 0.989


In [40]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [41]:
symptom_gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

In [42]:
model_gb = symptom_gb.fit(df_symptoms.iloc[:, 2:-1], df_symptoms['R'])

In [43]:
y_pred = model_gb.predict(df_symptoms.iloc[:, 2:-1])

In [44]:
precision, recall, fscore, train_support = score(df_symptoms['R'], y_pred, pos_label=1, average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==df_symptoms['R']).sum()/len(y_pred), 3)))

Precision: 0.9 / Recall: 0.226 / Accuracy: 0.989
